# COMPASS Quick Start

This notebook demonstrates the basic COMPASS workflow:

1. Load a YAML configuration for a 2x2 pixel unit cell
2. Build a solver-agnostic `PixelStack` from the config
3. Generate layer slices for the solver
4. Run a single-wavelength QE calculation
5. Visualize the pixel structure cross-section

In [ ]:
from pathlib import Path

import numpy as np

from compass.core.config_schema import load_config
from compass.geometry.builder import GeometryBuilder
from compass.geometry.pixel_stack import PixelStack
from compass.materials.database import MaterialDB

## 1. Load Configuration

COMPASS uses a single YAML file to define the entire pixel structure, source, and solver settings.
The config is validated with Pydantic on load.

In [ ]:
# Load a default BSI (back-side illuminated) 1um pixel config
config_path = Path("../configs/pixel/default_bsi_1um.yaml")
config = load_config(config_path)

print(f"Pixel pitch: {config.pixel.pitch_x} x {config.pixel.pitch_y} um")
print(f"Unit cell: {config.pixel.array_nx} x {config.pixel.array_ny} pixels")
print(f"Number of layers: {len(config.pixel.layers)}")

## 2. Build the PixelStack

The `PixelStack` is the solver-agnostic representation of the pixel structure.
It holds geometry, material assignments, and coordinate grids.
All internal units are in **micrometers (um)**.

In [ ]:
# Initialize the material database (built-in + any custom CSV files)
mat_db = MaterialDB()

# Build the pixel stack from config
builder = GeometryBuilder(config.pixel, mat_db)
pixel_stack = builder.build()

print(f"Stack z-range: [{pixel_stack.z_min:.3f}, {pixel_stack.z_max:.3f}] um")
print(f"Total thickness: {pixel_stack.total_thickness:.3f} um")
print(f"Number of layer slices: {len(pixel_stack.layer_slices)}")

## 3. Generate Layer Slices

Each `LayerSlice` contains the permittivity distribution for one z-slice of the stack.
These are the inputs that RCWA and FDTD solvers consume.

In [ ]:
# Inspect individual layer slices
for i, layer in enumerate(pixel_stack.layer_slices):
    print(f"Layer {i}: z=[{layer.z_start:.3f}, {layer.z_end:.3f}] um, "
          f"thickness={layer.thickness:.3f} um, "
          f"name='{layer.name}'")

In [ ]:
# Get the permittivity grid for a specific layer at 550 nm
wavelength = 0.550  # um
layer_0 = pixel_stack.layer_slices[0]
eps_grid = layer_0.get_permittivity(wavelength, mat_db)

print(f"Permittivity grid shape: {eps_grid.shape}")
print(f"Grid represents {config.pixel.array_nx * config.pixel.pitch_x} x "
      f"{config.pixel.array_ny * config.pixel.pitch_y} um unit cell")

## 4. Run a Single-Wavelength QE Calculation

We use the `torcwa` RCWA solver for a single wavelength at normal incidence.
The solver returns `SimulationResult` which contains reflectance, transmittance,
absorption per layer, and field distributions.

In [ ]:
from compass.analysis.qe_calculator import compute_qe
from compass.solvers.base import SolverFactory
from compass.sources.planewave import PlaneWave

# Create solver instance
solver = SolverFactory.create("torcwa", config.solver)

# Define illumination: normal incidence plane wave at 550 nm
source = PlaneWave(wavelength=0.550, theta=0.0, phi=0.0, polarization="unpolarized")

# Run the simulation
result = solver.solve(pixel_stack, source)

print(f"Reflectance:   R = {result.reflectance:.4f}")
print(f"Transmittance: T = {result.transmittance:.4f}")
print(f"Absorption:    A = {result.absorption:.4f}")
print(f"Energy check:  R + T + A = {result.reflectance + result.transmittance + result.absorption:.6f}")

In [ ]:
# Compute QE for each pixel in the unit cell
qe_per_pixel = compute_qe(result, pixel_stack)

print("QE per pixel (2x2 unit cell):")
for px in range(config.pixel.array_nx):
    for py in range(config.pixel.array_ny):
        print(f"  Pixel ({px},{py}): QE = {qe_per_pixel[px, py]:.4f}")

## 5. Visualize the Pixel Structure

COMPASS provides built-in 2D cross-section plots to inspect the pixel geometry.

In [ ]:
from compass.visualization.structure_plot_2d import plot_structure_xz

# Plot XZ cross-section through the center of the unit cell
fig, ax = plot_structure_xz(
    pixel_stack,
    y_cut=config.pixel.pitch_y / 2,  # cut through pixel center
    wavelength=0.550,
    mat_db=mat_db,
    show_materials=True,
    figsize=(10, 6),
)
ax.set_title("BSI Pixel Cross-Section (XZ plane)")

## Summary

This notebook covered the core COMPASS workflow:

- **Config** -> `load_config()` validates YAML with Pydantic
- **Geometry** -> `GeometryBuilder` produces a `PixelStack` with `LayerSlice` objects
- **Solver** -> `SolverFactory.create()` returns a solver; `.solve()` returns `SimulationResult`
- **Analysis** -> `compute_qe()` extracts per-pixel quantum efficiency
- **Visualization** -> `plot_structure_xz()` renders cross-sections

Next: See `02_structure_visualization.ipynb` for advanced structure plotting.